# Entropy Decision Tree

In [ ]:
def entropy(target_col):
    elements, cnt = np.unique(target_col, return_counts = True)
    entropy = -np.sum([(cnt[i] / np.sum(cnt))*np.log2(cnt[i] / np.sum(cnt)) for i in range(len(elements))])
    return entropy

def InfoGain(data, split_attribute_name, target_name):
    total_entropy = entropy(data[target_name])
    
    vals, cnt = np.unique(data[split_attibute_name], return_counts = True)
    Weighted_Entropy = np.sum([(cnt[i] / np.sum(cnt))*entropy(data.where(data[split_attribute_name] == vals[i]).dropna()[target_name])
                              for i in range((len(vals)))])
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

def ID3(data, original_data, features, target_attribute_name, parent_node_class = None):
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    else:
        parent_node_class = np.unique(data[target_attribute_name])\
                            [np.argmax(np.unique(data[target_attribute_name], return_counts = True))]
        item_values = [InfoGain(data, feature, target_attribute_name) for feature in features]
        best_feature_index = np.argmax(item_values)
        best_feature = featuers[best_feature_index]
        
        tree = {best_feature : {}}
        features = [i for i in features if i != best_feature]
        
        for value in np.unique(data[best_feature]):
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = ID3(sub_data, data, features, target_attribute_name, parent_node_class)
            tree[best_feature][value] = subtree
        return (tree)

# Gini Decision Tree

In [ ]:
def gini(x):
    return 1 - ((x / x.sum())**2).sum()

def dt_gini(df):
    G_lst, G1_lst, G2_lst, cp_lst = [], [], [], []
    for feature in features:
        df.sort_values(by = feature, inplace = True)
        df.reset_index(drop = True, inplace = True)
        cut_index = []
        for i in range(len(df) - 1):
            if df["target"][i] != df["target"][i + 1]:
                cut_index.append(i)
            else:
                pass
        for idx in cut_index:
            cp = (df[feature][idx] + df[featuer][idx+1]) / 2
            df_cp1 = df[df[feature] <= cp]
            df_cp2 = df[df[feature] > cp]
            N, N1, N2 = len(df), len(df_cp1), len(df_cp2)
            G1, G2 = gini(df_cp1.target.value_counts()), gini(df_cp2.target.value_counts())
            G = G1*(N1/N) + G2*(N2/N)
            G_lst.append(G)
            G1_lst.append(G1)
            G2_lst.append(G2)
            cp_lst.append(cp)
        
        num_cp.append(len(cut_index))
    
    key = np.argmin(G_lst)
    
    if key < num_cp[0]:
        return featuers[0], cp_lst[key], min(G1_lst)
    elif num_cp[0] <= key < sum(num_cp[:2]):
        return featuers[1], cp_lst[key], min(G1_lst)
    elif sum(num_cp[:2]) <= key < sum(num_cp[:3]):
        return features[2], cp_lst[key], min(G1_lst)
    elif sum(num_cp[:3]) <= key < min(G1_lst):
        return features[3], cp_lst[key], min(G1_lst)

# HardVoting vs SoftVoting

In [ ]:
model_1 = [0.6, 0.4]
model_2 = [0.49, 0.51]
model_3 = [0.49, 0.51]
model_4 = [0.49, 0.51]

models = [model_1, model_2, model_3, model_4]

def hard_voting(models):
    class_0, class_1 = 0, 0
    for model in models:
        if model[0] < model[1]:
            class_1 += 1
        else:
            class_0 += 1
    if class_1 > class_0:
        print("class 1")
    else:
        print("class 0")

def soft_voting(models):
    class_0, class_1 = 0, 0
    for model in models:
        if model[0] < model[1]:
            class_1 += model[1]
        else:
            class_0 += model[0]
    if class_1 > class_0:
        print("class 1")
    else:
        print("class 0")